In [36]:
import pandas as pd

# Read the original CSV
df = pd.read_csv("input_data/sanitation_type.csv")

# Define the mapping
column_mapping = {
    "fid": "id",
    "Latitude": "lat",
    "Longitude": "long",
    "Toilets wi": "toilet_type_name",
    "Type": "toilet_type_id",
    "Descpt": "toilet_category_name",
    "Category": "toilet_category_id",
    "Region_Nam": "region_name",
    "Dist_Nam": "district_name",
    "Ward_Nam": "ward_name",
    "Village_Na": "village_name"
}

# Select and rename only the mapped columns
df_mapped = df[list(column_mapping.keys())].rename(columns=column_mapping)

# Save to new CSV
df_mapped.to_csv("derived_data/sanitation_type_with_population.csv", index=False)


✅ Standardized sanitation data: derived_data/sanitation_type_with_population.csv (279934 rows)


In [33]:
import pandas as pd

# --- 1) Configuration object (list of dicts with name + id + efficiency) ---
TOILET_CATEGORY_CONFIG = [
    {"name": "Septic Tank - 80",      "id": 1, "efficiency": 0.8},
    {"name": "Pit Latrine - 20",      "id": 2, "efficiency": 0.2},
    {"name": "Septic Tank sewer - 90","id": 3, "efficiency": 0.9},
    {"name": "Open defecation - 0",   "id": 4, "efficiency": 0.0},
]

# Build a lookup dict (id → efficiency)
EFF_BY_ID = {item["id"]: item["efficiency"] for item in TOILET_CATEGORY_CONFIG}

# --- 2) Load data ---
path = "derived_data/sanitation_type_with_population.csv"
df = pd.read_csv(path)

# Ensure toilet_category_id is numeric
df["toilet_category_id"] = pd.to_numeric(df["toilet_category_id"], errors="coerce")

# --- 3) Map efficiency to new column ---
df["pathogen_containment_efficiency"] = df["toilet_category_id"].map(EFF_BY_ID)


# --- 4) Add household population (assumed uniform = 10 people) ---
df["household_population"] = 10

# --- 5) Save and preview ---
df.to_csv(path, index=False)  # overwrite same file

In [34]:
import pandas as pd
import os

# Paths
src = "derived_data/sanitation_type_with_population.csv"
dst = "derived_data/net_pathogen_load_from_households.csv"
os.makedirs(os.path.dirname(dst), exist_ok=True)

# EFIO_DEFAULT: Based on human feces ~128 g/day × ~10⁸ CFU/g → ~1.28×10¹⁰ CFU/person/day
EFIO_DEFAULT = 1.28e10  # shedding rate used for L = Pop × EFIO × (1 – η)

df = pd.read_csv(src)

df["household_population"] = pd.to_numeric(df["household_population"], errors="coerce").fillna(0)
df["pathogen_containment_efficiency"] = pd.to_numeric(
    df.get("pathogen_containment_efficiency", 0), errors="coerce"
).fillna(0)

# Compute η and net load (L)
df["fio_load"] = df["household_population"] * EFIO_DEFAULT * (1 - df["pathogen_containment_efficiency"])

df.to_csv(dst, index=False)

In [ ]:
import pandas as pd
import os

# Input paths
priv_src = "input_data/private_boreholes.csv"
gov_src  = "input_data/government_boreholes.csv"

# Output paths (new files, originals remain unchanged)
priv_out = "derived_data/private_boreholes_with_id.csv"
gov_out  = "derived_data/government_boreholes_with_id.csv"
os.makedirs("derived_data", exist_ok=True)

def add_ids_copy(path_in, path_out, prefix):
    df = pd.read_csv(path_in)

    # Add id if missing, else move it to first column
    if "id" not in df.columns:
        df.insert(0, "id", [f"{prefix}_{i+1}" for i in range(len(df))])
    else:
        id_col = df.pop("id")
        df.insert(0, "id", id_col)

    df.to_csv(path_out, index=False)
    print(f"✅ Saved {path_out} with {len(df)} rows (id column first)")
    return df

# Generate new copies
priv = add_ids_copy(priv_src, priv_out, "privbh")
gov  = add_ids_copy(gov_src,  gov_out,  "govbh")

In [ ]:
import os
import numpy as np
import pandas as pd

# ---------- Paths ----------
toilets_src = "derived_data/net_pathogen_load_from_households.csv"   # must have id, lat, long, fio_load
priv_src    = "derived_data/private_boreholes_with_id.csv"
gov_src     = "derived_data/government_boreholes_with_id.csv"
dst         = "derived_data/net_surviving_pathogen_load_links.csv"
os.makedirs(os.path.dirname(dst), exist_ok=True)

# ---------- Config ----------
KS_PER_M = 0.001  # m^-1 (decay constant, adjust per environment)

RADIUS_BY_TYPE = {
    "private": 30,      # cutoff radius for private boreholes (m)
    "government": 100,   # cutoff radius for government boreholes (m)
}

# ---------- Helpers ----------
def require_cols(df: pd.DataFrame, name: str, cols=("id", "lat", "long")):
    missing = [c for c in cols if c not in df.columns]
    if missing:
        raise ValueError(f"{name}: missing required column(s): {missing}")

def haversine_m(lat1, lon1, lat2, lon2):
    R = 6371000.0
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat, dlon = lat2 - lat1, lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2
    return 2*R*np.arcsin(np.minimum(1.0, np.sqrt(a)))

def link_within_radius(toilets_df, bores_df, borehole_type: str, radius_m: float):
    records = []
    tgt_lat = bores_df["lat"].to_numpy(float)
    tgt_lon = bores_df["long"].to_numpy(float)
    tgt_id  = bores_df["id"].to_numpy()

    for _, row in toilets_df.iterrows():
        d = haversine_m(row["lat"], row["long"], tgt_lat, tgt_lon)
        mask = d <= radius_m
        if not np.any(mask):
            continue
        contrib = row["fio_load"] * np.exp(-KS_PER_M * d[mask])
        for bid, dist, surv in zip(tgt_id[mask], d[mask], contrib):
            records.append({
                "toilet_id": row["id"],
                "toilet_lat": row["lat"],
                "toilet_long": row["long"],
                "borehole_id": bid,
                "borehole_type": borehole_type,
                "distance_m": float(dist),
                "surviving_fio_load": float(surv),
            })
    return pd.DataFrame.from_records(records)

# ---------- Load ----------
toilets = pd.read_csv(toilets_src)
priv    = pd.read_csv(priv_src)
gov     = pd.read_csv(gov_src)

require_cols(toilets, "toilets", ("id", "lat", "long", "fio_load"))
require_cols(priv, "private_boreholes")
require_cols(gov,  "government_boreholes")

# ---------- Build link table ----------
links_private = link_within_radius(toilets, priv, "private", RADIUS_BY_TYPE["private"])
links_government = link_within_radius(toilets, gov, "government", RADIUS_BY_TYPE["government"])

links = pd.concat([links_private, links_government], ignore_index=True)

# ---------- Save + preview ----------
links.to_csv(dst, index=False)

In [ ]:
import os
import numpy as np
import pandas as pd

# ---------- Paths ----------
links_src = "derived_data/net_surviving_pathogen_load_links.csv"  # from Layer 2 (link table)
priv_src  = "derived_data/private_boreholes_with_id.csv"          # has 'id' + coords (+ optional flow cols)
gov_src   = "derived_data/government_boreholes_with_id.csv"
links_out = "derived_data/net_surviving_pathogen_concentration_links.csv"
bh_out    = "derived_data/fio_concentration_at_boreholes.csv"
os.makedirs(os.path.dirname(links_out), exist_ok=True)

# ---------- Config ----------
# If borehole files have any of these columns, we'll compute Q (L/day) from them.
# Priority order: first present/non-null wins.
FLOW_COLUMN_PREFERENCE = [
    # already in L/day
    ("Q_L_per_day",          "L_per_day", 1.0),
    ("flow_L_per_day",       "L_per_day", 1.0),

    # in L/s -> convert to L/day (× 86400)
    ("discharge_Lps",        "L_per_s",   86400.0),
    ("Q_Lps",                "L_per_s",   86400.0),

    # in m^3/day -> convert to L/day (× 1000)
    ("yield_m3_per_day",     "m3_per_day", 1000.0),
    ("Q_m3_per_day",         "m3_per_day", 1000.0),
]

# Type-specific default Q if not present in the files (units: L/day)
Q_DEFAULT_BY_TYPE = {
    "private":    2_000.0,   # example placeholder; adjust to your context
    "government": 20_000.0,  # example placeholder; adjust to your context
}

# ---------- Helpers ----------
def coerce_num(s):
    return pd.to_numeric(s, errors="coerce")

def pick_Q_L_per_day(df: pd.DataFrame) -> pd.Series:
    """
    Return a Series of Q (L/day) from the borehole dataframe:
    - tries preferred columns (with unit conversions)
    - returns NaN if nothing found
    """
    q = pd.Series(np.nan, index=df.index, dtype="float64")
    for col, kind, factor in FLOW_COLUMN_PREFERENCE:
        if col in df.columns:
            vals = coerce_num(df[col])
            q = q.fillna(vals * factor)  # only fill where q is NaN
    return q

def build_borehole_Q_map(df: pd.DataFrame, borehole_type: str) -> pd.DataFrame:
    if "id" not in df.columns:
        raise ValueError(f"{borehole_type} boreholes: missing 'id' column.")

    q = pick_Q_L_per_day(df)
    # fallback to default by type if Q still missing
    q = q.fillna(Q_DEFAULT_BY_TYPE[borehole_type])

    out = df[["id"]].copy()
    out["borehole_type"] = borehole_type
    out["Q_L_per_day"] = coerce_num(q)
    return out

# ---------- Load ----------
links = pd.read_csv(links_src)   # needs: borehole_id, borehole_type, surviving_fio_load
priv  = pd.read_csv(priv_src)
gov   = pd.read_csv(gov_src)

required_link_cols = ["borehole_id", "borehole_type", "surviving_fio_load"]
missing = [c for c in required_link_cols if c not in links.columns]
if missing:
    raise ValueError(f"links file missing: {missing}")

# ---------- Build Q lookup ----------
priv_q = build_borehole_Q_map(priv, "private")
gov_q  = build_borehole_Q_map(gov,  "government")
bh_q   = pd.concat([priv_q, gov_q], ignore_index=True)

# ---------- Join Q and compute concentration per link ----------
links["surviving_fio_load"] = coerce_num(links["surviving_fio_load"]).fillna(0)
links = links.merge(bh_q.rename(columns={"id": "borehole_id"}), on=["borehole_id", "borehole_type"], how="left")

if links["Q_L_per_day"].isna().any():
    # If any still NaN (e.g., borehole type label mismatch), raise to avoid silent errors
    bad = links[links["Q_L_per_day"].isna()][["borehole_id", "borehole_type"]].drop_duplicates()
    raise ValueError(f"Missing Q for some boreholes:\n{bad}")

# C = L_t / Q
links["concentration_CFU_per_L"] = links["surviving_fio_load"] / links["Q_L_per_day"]

# ---------- Save link-level concentrations ----------
links.to_csv(links_out, index=False)
print(f"✅ Wrote link-level concentrations: {links_out}")
print(links[["toilet_id","borehole_id","borehole_type","surviving_fio_load","Q_L_per_day","concentration_CFU_per_L"]].head(8))

# ---------- Aggregate per borehole ----------
agg = (
    links
    .groupby(["borehole_id", "borehole_type"], as_index=False)
    .agg(
        total_surviving_fio_load=("surviving_fio_load", "sum"),
        Q_L_per_day=("Q_L_per_day", "first"),  # same Q per borehole
    )
)
agg["concentration_CFU_per_L"] = agg["total_surviving_fio_load"] / agg["Q_L_per_day"]

agg.to_csv(bh_out, index=False)
print(f"✅ Wrote borehole-level concentrations: {bh_out}")
print(agg.head(8))